<a class="anchor" id="top"></a>

# 1. Importing Libraries & Data
In this section, we set up the foundation for our project by importing the necessary Python libraries and loading the dataset. These libraries provide the tools for data manipulation, visualization, and machine learning modeling throughout the notebook. Additionally, we import the historical claims dataset, which forms the core of our analysis. 

In [73]:
import pandas as pd
import numpy as np


import utils as u

import matplotlib.pyplot as plt
import seaborn as sns
import viz as v

%matplotlib inline

pd.set_option('display.max_columns', None)
# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

**Import Data**

In [74]:
# Load training data
df = pd.read_csv('./project_data/train_data.csv', index_col = 'Claim Identifier')

# Load testing data
test = pd.read_csv('./project_data/test_data.csv', index_col = 'Claim Identifier')

# Display the first 3 rows of the training data
df.head(3)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,2020-02-21,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0


# 2. Exploratory Data Analysis
This section is dedicated to conducting an in-depth Exploratory Data Analysis (EDA) to comprehend the dataset’s structure, statistical properties, and inherent patterns. Through systematic analysis, we aim to identify key relationships among variables, detect potential anomalies, and evaluate the completeness and quality of the data. The findings from this stage serve as a critical foundation for subsequent data preprocessing and model development, ensuring that informed and data-driven decisions guide the overall analytical workflow.

<a href="#top">Top &#129033;</a>

**Profile Report**

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(
    df, 
    title='Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)

profile

From using a `Profile Report` we were able to identify some possible problems about variables:
- WCB Decision has constant value "Not Work Related"
- Alternative Dispute Resolution is highly imbalanced (97.3%)
- C-3 Date has 406226 (68.4%) missing values
- First Hearing Date has 442673 (74.6%) missing values
- IME-4 Count has 460668 (77.6%) missing values
- OIICS Nature of Injury Description has 593471 (100.0%) missing values
- Zip Code is an unsupported type, check if it needs cleaning or further analysis
- Average Weekly Wage has 335450 (56.5%) zeros
- Birth Year has 25081 (4.2%) zeros
- Claim Injury Type (Target) has 19445 (3.3%) missing values and is imbalanced

And about the dataset itself:
- Dataset has 1103 (0.2%) duplicate rows

The same method was applied to test data

In [ ]:
profile_test = ProfileReport(
    test, 
    title='Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)

profile_test

From using a `Profile Report` for the test data we were able to identify some possible problems about variables:
- Alternative Dispute Resolution is highly imbalanced (97.5%)
- COVID-19 Indicator is highly imbalanced (94.3%)
- First Hearing Date has 344947 (88.9%) missing values
- IME-4 Count has 352726 (90.9%) missing values
- OIICS Nature of Injury Description has 387975 (100.0%) missing values

And about the test dataset itself: 
- Dataset has 2 (< 0.1%) duplicate rows

There are also columns that are present in the train data but not in the test data (apart from our Target variable):
- Agreement Reached
- WCB Decision

After using `Profile Report`, we will also employ well known methods for basic statistics, such as `describe`, `info` and `shape`, as well as print the unique values for each column.

In [ ]:
# The "include='object'" argument limits the description to only categorical data types.
# Transpose the result (.T) to display statistics in a more readable format with rows as columns.
df.describe(include='object').T

In [ ]:
# .describe provides statistical summaries for the numerical columns
df.describe().T

In [ ]:
# Display summary information about the DataFrame
df.info()

In [ ]:
# Check the dimensions of the dataset (number of rows and columns)
df.shape

In [ ]:
# Loop through each column in the DataFrame
for column in df.columns:
    
    # Check if the number of unique values in the column is less than 20
    if df[column].nunique() < 20:
        
        # Print unique values and count for columns with fewer than 20 unique values
        print(f"Unique values in '{column}': {df[column].unique()}")
        print(f"Number of unique values: {df[column].nunique()}\n")

From these, we were able to conclude that the dataset had 593471 rows and 32 columns. 

Additionally, we noticed that:
- WCB Decision only had 1 unique value
- We have 4 different values for Gender
- Age ranges from 0 to 117
- Birth Year ranges from 0 to 2018


There is also a column which is always null, **OIICS Nature of Injury Description**.

## 2.1 Variable Exploration

Incoherencies, can compromise the integrity of the analysis and the performance of machine learning models. By systematically identifying and addressing these issues, we aim to enhance data quality and ensure the dataset is both accurate and reliable for subsequent preprocessing and modeling stages.

<a href="#top">Top &#129033;</a>

In this section we will start by dropping the features that exist in the train data (df) but not in the test, as well **OIICS Nature of Injury Description**, which is always null.

We will also correct the date datatypes in both df and test.

Then, we will individually analyze each variable to better understand it.

In [75]:
columns = ['Agreement Reached', 'WCB Decision', 'OIICS Nature of Injury Description']

In [76]:
df = df.drop(columns = columns, axis = 1)
test = test.drop(columns = columns[-1], axis = 1)

In [77]:
# List of columns to convert to datetime
date_columns = ['Accident Date', 'Assembly Date', 'C-2 Date', 'C-3 Date', 'First Hearing Date']

# Apply pd.to_datetime() to each column in the list for both df and test
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    test[col] = pd.to_datetime(test[col], errors='coerce')


Having made these initial transformations, we will start the exploration by comparing the minimum, average and maximum values of both df and test datasets, for all numeric variables

In [82]:
num_columns = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]

obj_columns = [col for col in df.columns if isinstance(df[col].dtype, pd.CategoricalDtype) or df[col].dtype == 'object']

In [83]:
num_stats_compare = u.num_stats(df, test, num_columns)
num_comparison_df = pd.DataFrame(num_stats_compare).T
num_comparison_df

,DF Mean,Test Mean,DF Std,Test Std,DF Min,Test Min,DF 25%,Test 25%,DF 50%,Test 50%,DF 75%,Test 75%,DF Max,Test Max
Age at Injury,42.114270,41.414944,14.256432,14.501056,0.0,0.0,31.0,30.0,42.0,40.0,54.0,53.0,117.0,114.0
Average Weekly Wage,491.088321,183.343831,6092.918120,3542.310214,0.0,0.0,0.0,0.0,0.0,0.0,841.0,0.0,2828079.0,1950317.0
Birth Year,1886.767604,1875.383466,414.644423,444.659075,0.0,0.0,1965.0,1967.0,1977.0,1980.0,1989.0,1992.0,2018.0,2019.0
IME-4 Count,3.207337,2.118131,2.832303,1.465406,1.0,1.0,1.0,1.0,2.0,2.0,4.0,2.0,73.0,18.0
Industry Code,58.645305,58.653581,19.644175,18.710501,11.0,11.0,45.0,45.0,61.0,61.0,71.0,71.0,92.0,92.0
WCIO Cause of Injury Code,54.381143,53.335678,25.874281,26.176833,1.0,1.0,31.0,29.0,56.0,56.0,75.0,74.0,99.0,99.0
WCIO Nature of Injury Code,41.013839,38.373674,22.207521,20.966100,1.0,1.0,16.0,10.0,49.0,43.0,52.0,52.0,91.0,91.0
WCIO Part Of Body Code,39.738146,31.516109,22.365940,23.359950,-9.0,-9.0,33.0,15.0,38.0,36.0,53.0,51.0,99.0,99.0
Number of Dependents,3.006559,3.000284,2.000801,1.997982,0.0,0.0,1.0,1.0,3.0,3.0,5.0,5.0,6.0,6.0


From this table, the main conclusions are:
- **Average Weekly Wage** mean values in DF (491) are much larger than in test (183)
- **Average Weekly Wage** values in the 75% percentile are much larger in df (841) than in test (0), which indicates a distribution shift between DF and test sets

Apart from these, there are no other significant differences in the numeric columns of both sets.

In [ ]:
obj_stats_compare = u.obj_stats(df, test, obj_columns)
obj_comparison_df = pd.DataFrame(obj_stats_compare).T
obj_comparison_df

From this table, the main conclusions are:
- **Carrier Type** has 8 unique values in DF, but only 7 in test
- **WCIO Part Of Body Description** has 54 unique values in DF, but only 51 in test
- **County of Injury** Mode is SUFFOLK in DF, but QUEENS in test
- **WCIO Cause of Injury Description** and **WCIO Part Of Body Description** have different modes from DF to test

### 2.1.1 Univariate Analysis

### Numerical Variables

In [ ]:
v.plot_numeric_histograms(df)

### Categorical Variables

**Carrier Type**

As we previously identified, there are 2 categories present in DF (5A, 5C) that do not exist in test. This can be confirmed by the 2 visualisations bellow

In [ ]:
v.plot_histogram_by_category(df, 'Carrier Type')

In [ ]:
v.plot_histogram_by_category(test, 'Carrier Type')

**WCIO Part Of Body Description**

In DF, even though teh category 'Multiple' is still common, there are 2 categories that are even more common ('Lower Back Area' and 'Knee'), when comparing to the test data.

In [ ]:
v.plot_histogram_by_category(df, 'WCIO Part Of Body Description', 
                          rotation = 90)

In [ ]:
v.plot_histogram_by_category(test, 'WCIO Part Of Body Description', 
                          rotation = 90)

### 2.1.2 Bivariate Analysis

### Numeric vs Numeric

In [ ]:
# Remove rows with missing values
temp = df.dropna()

# Select numerical columns for correlation analysis
corr_data = temp.select_dtypes(include=['number'])

# Compute the Spearman correlation matrix (Pearson is the default)
correlation_matrix = corr_data.corr(method='spearman')

# Plot the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='YlOrBr', linewidths=0.1)
plt.show()

From plotting a correlation matrix for numeric features, we can conclude that there are no highly positive correlations. Nonetheless, there is a highly negative correlation (-0.91) between **Birth Year** and **Age at Injury**. 

This negative correlation was to be expected, as the age can be computed from the birth year, and the higher the birth year, the lower the age.

This problem will be addressed later in the project.

In [ ]:
def plot_pairwise_relationship(df):
    # Select only numerical columns
    num_columns = df.select_dtypes(include=['number']).columns

    # Create a pairplot with only the lower triangle
    g = sns.pairplot(df[num_columns], kind='scatter', 
                 hue=None, plot_kws={'s': 10, 'color': 'orange'}, 
                 corner=True) 
    
    # Update the diagonal plots to be orange
    for ax in g.diag_axes:
        for patch in ax.patches:
            patch.set_facecolor('orange')

        
    plt.show()

In [ ]:
plot_pairwise_relationship(df)

Above we are plotting scatterplots for all combinations of numeric variables, looking for linear relationships between them. Since this matrix is mirrored on the diagonal, we decided to only display half of it.

This allowed us to conclude that there are no linear relationships when combining two numeric variables.

**Numeric vs Objects**

In [ ]:
discrete = ['Birth Year', 'IME-4 Count', 'Industry Code',
            'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code',
            'WCIO Part Of Body Code', 'Number of Dependents']

continuous = ['Age at Injury', 'Average Weekly Wage']

objects = [col for col in df.select_dtypes(include=['object']).columns
               if df[col].nunique() < 9]

For Categorical vs Numerical Discrete Variables

In [ ]:
# def plot_categorical_vs_discrete_matrix(df, categorical, discrete, n_cols=3):
#     # Calculate the number of rows needed based on the number of categories and columns
#     n_cats = len(categorical)
#     n_discrete = len(discrete)
    
#     # Calculate number of rows required
#     n_rows = int(np.ceil(n_cats * n_discrete / n_cols))
    
#     # Create a figure with subplots
#     fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(n_cols * 6, n_rows * 6))
    
#     # Flatten axes for easy iteration
#     axes = axes.flatten()

#     plot_idx = 0
#     for i, cat in enumerate(categorical):
#         for j, disc in enumerate(discrete):
#             if plot_idx < len(axes):
#                 # Plot the countplot for each combination of categorical and discrete variables
#                 sns.countplot(x=disc, hue=cat, data=df, palette='Oranges', ax=axes[plot_idx])
#                 axes[plot_idx].set_title(f'{cat} vs {disc}')
#                 axes[plot_idx].tick_params(axis='x', rotation=45)
#                 plot_idx += 1
                
                
#                 # Hide x-tick labels if there are more than 8
#                 if len(df[column].value_counts()) > 8:
#                     axes.set_xticklabels([])
#                 else:
#                     axes.tick_params(axis='x', rotation=45)

#     # Hide any unused subplots
#     for idx in range(plot_idx, len(axes)):
#         axes[idx].axis('off')

#     # Adjust layout for better spacing
#     plt.tight_layout()
#     plt.show()

In [ ]:
# plot_categorical_vs_discrete_matrix(df, categorical, discrete, n_cols=3)

For Categorical vs Numerical Continuous Variables

In [ ]:
def plot_categ_cont(df, categorical, continuous, n_cols=3):
    # Calculate the number of rows needed based on number of categories and columns
    n_cats = len(categorical)
    n_cont = len(continuous)
    
    # Calculate number of rows required
    n_rows = int(np.ceil(n_cats / n_cols))
    
    # Create a figure with subplots
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(n_cols * 6, n_rows * 6))
    
    # Flatten axes for easy iteration
    axes = axes.flatten()

    plot_idx = 0
    for i, cat in enumerate(categorical):
        for j, cont in enumerate(continuous):
            if plot_idx < len(axes):
                sns.boxplot(x=cat, y=cont, data=df, palette='Oranges', ax=axes[plot_idx])
                axes[plot_idx].set_title(f'{cat} vs {cont}')
                axes[plot_idx].tick_params(axis='x', rotation=45)
                plot_idx += 1

    # Hide any unused subplots
    for idx in range(plot_idx, len(axes)):
        axes[idx].axis('off')

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

In [ ]:
plot_categ_cont(df, categorical, continuous)

**Categorical vs Categorical**

In [ ]:
# def plot_crosstab_matrix(df, categorical, n_cols=3):
#     n = len(categorical)

#     # Calculate the number of rows needed based on the number of columns specified
#     n_rows = (n * (n - 1)) // n_cols  # Ceiling division to ensure proper number of rows

#     # Create a figure with subplots, n_rows x n_cols grid
#     fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))
#     axes = axes.flatten()  # Flatten the axes to easily index them

#     # Loop through all pairs of categorical columns (col1, col2)
#     ax_idx = 0  # This will help us to assign axes for each plot
#     for i, col1 in enumerate(categorical):
#         for j, col2 in enumerate(categorical):
#             if col1 != col2:
#                 # Create crosstab between col1 and col2
#                 ct = pd.crosstab(df[col1], df[col2])

#                 # Ensure there are enough subplots, if not break
#                 if ax_idx >= len(axes):
#                     break

#                 # Create heatmap for this pair of categorical columns
#                 sns.heatmap(ct, annot=True, fmt='d', cmap='Blues', ax=axes[ax_idx], cbar=False)
#                 axes[ax_idx].set_title(f'{col1} vs {col2}')
#                 axes[ax_idx].tick_params(axis='x', rotation=45)
#                 axes[ax_idx].tick_params(axis='y', rotation=0)
#                 ax_idx += 1  # Move to the next axis

#     # Hide unused subplots if there are any remaining empty axes
#     for i in range(ax_idx, len(axes)):
#         axes[i].axis('off')

#     # Adjust layout to prevent overlap and display the plots
#     plt.tight_layout()
#     plt.show()

In [ ]:
# plot_crosstab_matrix(df, categorical)

# 3. Data Cleaning & Preprocessing
The cleaning process involves handling missing values, correcting errors, and addressing inconsistencies identified earlier. Preprocessing includes tasks such as feature engineering, encoding categorical variables, normalizing numerical data, and splitting the data into training and testing sets. These steps are essential to ensure that the dataset is both structured and suitable for developing robust and reliable machine learning models.

<a href="#top">Top &#129033;</a>

**Claim Injury Type**

Having noted that in DF there are missing values in this variable, we must address them before continuing. Since this is our target, when missing the entire observation cannot be used for training purposes, therefore must be dropped.

In [78]:
# Filter and display rows where the 'Claim Injury Type' column has missing values
df[df['Claim Injury Type'].isna()]

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
957648180,NaT,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363651794,NaT,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209507552,NaT,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970865999,NaT,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807753247,NaT,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365870975,NaT,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327160035,NaT,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249875936,NaT,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# Remove rows with missing values in the 'Claim Injury Type' column
df.dropna(subset=['Claim Injury Type'], inplace=True)

Having treated our target, it must also be encoded.

In [80]:
# Map descriptive labels in 'Claim Injury Type' to numeric codes for consistency and modeling
label_mapping = {
    "1. CANCELLED": 0,
    "2. NON-COMP": 1,
    "3. MED ONLY": 2,
    "4. TEMPORARY": 3,
    "5. PPD SCH LOSS": 4,
    "6. PPD NSL": 5,
    "7. PTD": 6,
    "8. DEATH": 7
}

In [81]:
df['Claim Injury Type'] = df['Claim Injury Type'].replace(label_mapping)

## 3.1 Duplicates
In this section, we will focus on detecting and handling duplicates to ensure data integrity and improve model performance. Addressing duplicates is a crucial preprocessing step that contributes to better data quality and more reliable machine learning outcomes.

<a href="#top">Top &#129033;</a>

In [82]:
# Identify and return rows that are duplicates in the DataFrame
df[df.duplicated()]

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5686771,NaT,0.0,N,2021-03-25,Y,0.0,1968.0,NaT,2021-03-16,ACE AMERICAN INSURANCE CO,1A. PRIVATE,0,UNKNOWN,N,NYC,2022-10-24,M,NaN,NaN,NaN,UK,NaN,NaN,NaN,NaN,NaN,NaN,07032,0.0


In [83]:
# Remove duplicate rows from the DataFrame
df = df.drop_duplicates()

## 3.2 Feature Engineering
Feature engineering is the process of transforming raw data into meaningful features to improve machine learning model performance. Effective feature engineering helps enhance predictive accuracy, reduce overfitting, and optimize model outcomes. In this section, we will explore various techniques and their impact on improving machine learning models.

<a href="#top">Top &#129033;</a>

### 3.2.1 Encoding

Encoding transforms categorical data into numerical format for use in machine learning models. For this section, several encoders were considered:
- **One Hot Encoding** -  turns a variable that is stored in a column into dummy variables stored over multiple columns and represented as 0s and 1s
- **Frequency Encoding** - replaces the categories by with their proportion in the dataset
- **Count Encoding** - replaces the categories by the number of times they appear in the dataset 
- **Manual Mapping Encoding** - manually attribute values to each category

<a href="#top">Top &#129033;</a>

**Alternative Dispute Resolution**

Knowing that 'N' is by far the most common category, and that 'U' only appears 5 times in DF data and 1 time in the test data, we decided to join 'U' and 'Y' into 'Y/U', and encode the variable as:
- 0 - N
- 1 - Y/U

In [84]:
print(df['Alternative Dispute Resolution'].value_counts())
print(' ')
print(test['Alternative Dispute Resolution'].value_counts())

Alternative Dispute Resolution
N    571411
Y      2609
U         5
Name: count, dtype: int64
 
Alternative Dispute Resolution
N    386314
Y      1660
U         1
Name: count, dtype: int64


In [85]:
df['Alternative Dispute Resolution Bin'] = df['Alternative Dispute Resolution'].replace({'N': 0, 'Y': 1, 'U': 1})
test['Alternative Dispute Resolution Bin'] = test['Alternative Dispute Resolution'].replace({'N': 0, 'Y': 1, 'U': 1})

**Attorney/Representative**

As this variable only has 2 categories, they will be encoded as follows:
- N - 0
- Y - 1

In [86]:
print(df['Attorney/Representative'].value_counts())
print(' ')
print(test['Attorney/Representative'].value_counts())

Attorney/Representative
N    392291
Y    181734
Name: count, dtype: int64
 
Attorney/Representative
N    306476
Y     81499
Name: count, dtype: int64


In [87]:
# Replace 'N' with 0 and 'Y' with 1 in the 'Attorney/Representative' column for binary encoding
df['Attorney/Representative Bin'] = df['Attorney/Representative'].replace({'N': 0, 'Y': 1})
test['Attorney/Representative Bin'] = test['Attorney/Representative'].replace({'N': 0, 'Y': 1})

**Carrier Name**

As Carrier name has a considerable amount of unique values, it will be encoded using Count Encoder.

In [88]:
# Get unique carrier names in train and test
train_carriers = set(df['Carrier Name'].unique())
test_carriers = set(test['Carrier Name'].unique())

# Find carriers in test that are not in train
test_only_carriers = test_carriers - train_carriers
train_only_carriers = train_carriers - test_carriers

# Display the result
print(f"Carrier Names in test but not in train: {test_only_carriers}")
print(f'Size of Carrier Names in test but not in train: {len(test_only_carriers)}')
print(' ')
print(f"Carrier Names in train but not in test: {train_only_carriers}")
print(f'Size of Carrier Names in train but not in test: {len(train_only_carriers)}')

Carrier Names in test but not in train: {'ULLICO CASUALTY COMPANY', 'NEW YORK TRANSPORTATION', 'EDMESTON CSD', 'FIRE & CASUALTY INS CO OF CONN', 'FULTON BOILER WORKS INC', 'ALCOA, INC. NKA ARCONIC, INC', 'FRANKLIN CENTRAL SCHOOL', 'WINDHAM-ASHLAND-JEWETT CSD', 'BERKSHIRE MUTUAL INS. CO.', 'T. MARZETTI COMPANY', 'CLYDE-SAVANNAH CENTRAL', 'ROOT PROPERTY & CASUALTY', 'TOWN OF PARMA', 'TOWER INSURANCE CO OF NY', 'CORINTH CSD', 'VILLAGE OF HOMER', 'EASTPORT UFSD', 'COREPOINTE INSURANCE CO', "NY MCDONALD'S OPER WC TRUST", 'TOWN OF ORCHARD PARK', 'ROADWAY EXPRESS INC', 'TRAVELERS PROPERTY CAS. INS.', 'PROJECT P HOLDINGS, LLC', 'CHENANGO CNTY. SELF-INS PLAN', 'EXPLORER INSURANCE COMPANY', 'NEW JERSEY CASUALTY INSURANCE', 'BOLTON CSD', 'MANUFACTURING SELF-INS TRUST', 'GRANVILLE CSD', 'PENNSYLVANIA NATIONAL MUTUAL', 'SHELTER ISLAND UFSD', 'NORTH POINTE INSURANCE CO.', 'FIRSTCOMP INSURANCE COMPANY', 'TOWN OF NEWBURGH', 'MONTAUK UNION FREE', 'BIRDS EYE FOODS INC', 'SHOPWELL INC', 'NATIONWIDE INSUR

In [89]:
common_categories = train_carriers.intersection(test_carriers)
len(common_categories)

1521

In [90]:
common_category_map = {category: idx + 1 for idx, category in enumerate(common_categories)}

def encode_category(carrier, carrier_map):
    return carrier_map.get(carrier, 0)

In [91]:
df['Carrier Name Enc'] = df['Carrier Name'].apply(lambda x: encode_category(x, common_category_map))
test['Carrier Name Enc'] = test['Carrier Name'].apply(lambda x: encode_category(x, common_category_map))


**Carrier Type**

DF contains 8 unique values, whereas test only contains 7. The difference is in '5C. SPECIAL FUND - POI CARRIER WCB MENANDS'. Knowing this, we decided to group all '5' categories into a single one: 
- '5. SPECIAL FUND'

In [92]:
print(df['Carrier Type'].value_counts())
print(' ')
print(test['Carrier Type'].value_counts())

Carrier Type
1A. PRIVATE                                    285367
3A. SELF PUBLIC                                121920
2A. SIF                                        111144
4A. SELF PRIVATE                                52788
UNKNOWN                                          1774
5D. SPECIAL FUND - UNKNOWN                       1023
5C. SPECIAL FUND - POI CARRIER WCB MENANDS          5
5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)         4
Name: count, dtype: int64
 
Carrier Type
1A. PRIVATE                                    201827
3A. SELF PUBLIC                                 78948
2A. SIF                                         66189
4A. SELF PRIVATE                                38619
UNKNOWN                                          1642
5D. SPECIAL FUND - UNKNOWN                        748
5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)         2
Name: count, dtype: int64


In [93]:
mapping = {
    '5D. SPECIAL FUND - UNKNOWN': '5. SPECIAL FUND',
    '5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A)': '5. SPECIAL FUND',
    '5C. SPECIAL FUND - POI CARRIER WCB MENANDS': '5. SPECIAL FUND'
}

In [94]:
# Apply the mapping to standardize 'Carrier Type' values
df['Carrier Type'] = df['Carrier Type'].replace(mapping)

test['Carrier Type'] = test['Carrier Type'].replace(mapping)

After doing so, we decided to encode them in 2 distinct ways, and choose the best option in feature selection.

Starting with **Count Encoding**

In [95]:
freq = df['Carrier Type'].value_counts()

df['Carrier Type freq'] = df['Carrier Type'].map(freq)
test['Carrier Type freq'] = test['Carrier Type'].map(freq)

And **One-Hot-Encoding**

In [96]:
# Convert the 'Carrier Type' column into one-hot encoded dummy variables
df = pd.get_dummies(df, columns=['Carrier Type'])
test = pd.get_dummies(test, columns=['Carrier Type'])

# Drop the last dummy variable to prevent multicollinearity (redundancy)
df = df.iloc[:, :-1]
test = test.iloc[:, :-1]

In [97]:
# Select columns related to 'Carrier Type' based on their column names
dummy_columns = df.filter(like='Carrier Type').columns

df[dummy_columns] = df[dummy_columns].astype(int)
test[dummy_columns] = test[dummy_columns].astype(int)

**County of Injury**

As County of Injury has a considerable amount of unique values, it will be encoded using Count Encoder.

In [98]:
# Count the frequency of each unique value in the 'County of Injury' column
freq = df['County of Injury'].value_counts()

# Map the frequency of each 'County of Injury' to replace values with their respective counts
df['County of Injury freq'] = df['County of Injury'].map(freq)
test['County of Injury freq'] = test['County of Injury'].map(freq)

**COVID-19 Indicator**

As this variable only has 2 categories, they will be encoded as follows:
- N - 0
- Y - 1

In [99]:
print(df['COVID-19 Indicator'].value_counts())
print(' ')
print(test['COVID-19 Indicator'].value_counts())

COVID-19 Indicator
N    546504
Y     27521
Name: count, dtype: int64
 
COVID-19 Indicator
N    385434
Y      2541
Name: count, dtype: int64


In [100]:
df['COVID-19 Indicator Enc'] = df['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})
test['COVID-19 Indicator Enc'] = test['COVID-19 Indicator'].replace({'N': 0, 'Y': 1})

**District Name**

As this variable has 8 unique values, Count Encoder will be used.

In [101]:
print(df['District Name'].value_counts())
print(' ')
print(test['District Name'].value_counts())

District Name
NYC           270778
ALBANY         86171
HAUPPAUGE      60536
BUFFALO        45605
SYRACUSE       44646
ROCHESTER      40449
BINGHAMTON     21864
STATEWIDE       3976
Name: count, dtype: int64
 
District Name
NYC           187972
ALBANY         56500
HAUPPAUGE      36656
BUFFALO        31481
SYRACUSE       29537
ROCHESTER      28073
BINGHAMTON     15382
STATEWIDE       2374
Name: count, dtype: int64


In [102]:
freq = df['District Name'].value_counts()

In [103]:
# Replace each district name in the 'District Name' column with its corresponding frequency count
df['District Name freq'] = df['District Name'].map(freq)
test['District Name freq'] = test['District Name'].map(freq)

**Gender**

As the number of 'X' and 'U' genders is very small compared with the others, they are going to be grouped and encoded as follows.
- M - 0
- F - 1
- U & X - U/X - 2

In [104]:
print(df['Gender'].value_counts())
print(' ')
print(test['Gender'].value_counts())

Gender
M    335217
F    234051
U      4711
X        46
Name: count, dtype: int64
 
Gender
M    215343
F    167019
U      5340
X       273
Name: count, dtype: int64


In [105]:
mapping = {  
    'M': 'M',
    'F': 'F',
    'U': 'U/X',  
    'X': 'U/X'
}

df['Gender'] = df['Gender'].map(mapping)  
test['Gender'] = test['Gender'].map(mapping) 

In [106]:
df['Gender Enc'] = df['Gender'].replace({'M': 0, 'F': 1, 'U/X': 2})
test['Gender Enc'] = test['Gender'].replace({'M': 0, 'F': 1, 'U/X': 2})

Since after grouping we only have 3 categories, we will also use **One-Hot-Encoding** and decide which is better in feature selection.

In [107]:
# Convert the 'Carrier Type' column into one-hot encoded dummy variables
df = pd.get_dummies(df, columns=['Gender'])
test = pd.get_dummies(test, columns=['Gender'])

# Drop the last dummy variable to prevent multicollinearity (redundancy)
df = df.iloc[:, :-1]
test = test.iloc[:, :-1]

In [108]:
# Select columns related to 'Carrier Type' based on their column names
dummy_columns = df.filter(like='Gender').columns

df[dummy_columns] = df[dummy_columns].astype(int)
test[dummy_columns] = test[dummy_columns].astype(int)

**Medical Fee Region**

Even though this variable only contains 5 unique values, it is not clear whether there is an order between them or not. Therefore we will use **Count Encoding**

In [109]:
print(df['Medical Fee Region'].value_counts())
print(' ')
print(test['Medical Fee Region'].value_counts())

Medical Fee Region
IV     265981
I      135885
II      85033
III     53654
UK      33472
Name: count, dtype: int64
 
Medical Fee Region
IV     182276
I       91300
II      58743
III     34679
UK      20977
Name: count, dtype: int64


In [110]:
freq = df['Medical Fee Region'].value_counts()

# Replace each 'Medical Fee Region' value with its corresponding frequency count
df['Medical Fee Region freq'] = df['Medical Fee Region'].map(freq)
test['Medical Fee Region freq'] = test['Medical Fee Region'].map(freq)

### 3.2.2 Feature Creation

<a href="#top">Top &#129033;</a>

**Date Columns**

For each date column, the Year, Month, Day and Week Day (0=Monday, 6=Sunday) will be extracted.

In [111]:
for column in df.columns:
    # Check if the column is a datetime type
    if pd.api.types.is_datetime64_any_dtype(df[column]) and column not in ['C-3 Date', 'First Hearing Date']:
        # Extract year, month, and day and create new columns
        df[f'{column} Year'] = df[column].dt.year
        df[f'{column} Month'] = df[column].dt.month
        df[f'{column} Day'] = df[column].dt.day
        df[f'{column} Day of Week'] = df[column].dt.weekday 
        
        test[f'{column} Year'] = test[column].dt.year
        test[f'{column} Month'] = test[column].dt.month
        test[f'{column} Day'] = test[column].dt.day
        test[f'{column} Day of Week'] = test[column].dt.weekday

**WCIO Codes**

All WCIO codes will be joined in a column. Before joining them, they will be transformed into integers. For this to be possible, missing values will be filled with a specific code, 0, which until now does not exist in any of the mentioned columns. We also needed to transform **WCIO Part Of Body Code** into the absolute value, since there was a negative code (-9). Before doing so, we ensured there was not any code with the same absolute number.

In [112]:
df['WCIO Part Of Body Code'] = df['WCIO Part Of Body Code'].abs()
test['WCIO Part Of Body Code'] = test['WCIO Part Of Body Code'].abs()

In [113]:
columns_to_join = [
    'WCIO Cause of Injury Code',
    'WCIO Nature of Injury Code',
    'WCIO Part Of Body Code'
]

df[columns_to_join] = df[columns_to_join].fillna(0).astype(int)
test[columns_to_join] = test[columns_to_join].fillna(0).astype(int)

df['WCIO Codes'] = df[columns_to_join].astype(str).agg(''.join, axis=1).astype(int)
test['WCIO Codes'] = test[columns_to_join].astype(str).agg(''.join, axis=1).astype(int)

**Zip Code**

Will be transformed into a Zip Code Valid, which evaluates the validity of the Zip Code:
- -1 for missing values
- 0 for a valid numeric value
- 1 for non-numeric zip codes

In [114]:
# Create a new column 'Zip Code Valid' to flag the validity of the 'Zip Code' field
df['Zip Code Valid'] = df['Zip Code'].apply(
    lambda x: -1 if pd.isna(x)          
    else (1 if not str(x).isnumeric()   
          else 0)                       
)


test['Zip Code Valid'] = test['Zip Code'].apply(
    lambda x: -1 if pd.isna(x)          
    else (1 if not str(x).isnumeric()   
          else 0)                       
)

**Industry Code Description**

The descriptions of the Industries will be grouped by sectors, as follows:
- Public Services / Government:
    - Public Administration 
    - Health Care and Social Assistance 
    - Educational Services 
    - Arts, Entertainment, and Recreation 
- Business Services:
    - Professional, Scientific, and Technical Services 
    - Administrative and Support and Waste Management and Remediation
    - Information 
    - Management of Companies and Enterprises 
    - Finance and Insurance 
- Retail and Wholesale:
    - Retail Trade
    - Wholesale Trade 
    - Accommodation and Food Services 
- Manufacturing and Construction:
    - Manufacturing 
    - Construction 
- Transportation:
    - Transportation and Warehousing 
- Agriculture and Natural Resources:
    - Agriculture, Forestry, Fishing, and Hunting 
    - Mining
- Utilities 
    - Utilities
- Other Services
    - Other Services (Except Public Services)

    
Then, they will be encoded.


In [115]:
df['Industry Sector'] = df['Industry Code Description'].apply(u.group_industry)
test['Industry Sector'] = test['Industry Code Description'].apply(u.group_industry)

Count Encoding

In [116]:
freq = df['Industry Sector'].value_counts()

In [117]:
df['Industry Sector Count Enc'] = df['Industry Sector'].map(freq)
test['Industry Sector Count Enc'] = test['Industry Sector'].map(freq)

**Age Groups**

Creating groups for age as follows:
- Minors (0): Ages 0-17
- Adults (1): Ages 18-64
- Seniors (2): Ages 65+

In [145]:
bins = [-1, 17, 64, 117]
labels = [0, 1, 2]

df['Age Group'] = pd.cut(df['Age at Injury'], bins=bins, labels=labels, right=True)

test['Age Group'] = pd.cut(test['Age at Injury'], bins=bins, labels=labels, right=True)

Drop treated and unnecessary columns before continuing.

In [119]:
# Columns to drop:
to_drop = ['Accident Date', 'Alternative Dispute Resolution', 'Assembly Date', 'Attorney/Representative',
              'C-2 Date', 'Carrier Name','County of Injury', 'COVID-19 Indicator', 'District Name',
              'Medical Fee Region', 'Zip Code', 'Industry Sector']

to_drop2 = ['WCIO Cause of Injury Description', 'WCIO Nature of Injury Description', 
              'WCIO Part Of Body Description', 'Industry Code Description']

all_drop = to_drop + to_drop2

In [120]:
df.drop(columns = all_drop, axis = 1, inplace = True)
test.drop(columns = all_drop, axis = 1, inplace = True)

# 4. Missing Values

<a href="#top">Top &#129033;</a>

Before spliting df into train and validation sets, we can fill missing values with simple methods (constants, median)

In [146]:
df.isna().sum()

Age at Injury                             0
Average Weekly Wage                   28651
Birth Year                            29078
Claim Injury Type                         0
IME-4 Count                               0
Industry Code                             0
WCIO Cause of Injury Code                 0
WCIO Nature of Injury Code                0
WCIO Part Of Body Code                    0
Number of Dependents                      0
Alternative Dispute Resolution Bin        0
Attorney/Representative Bin               0
Carrier Name Enc                          0
Carrier Type freq                         0
Carrier Type_1A. PRIVATE                  0
Carrier Type_2A. SIF                      0
Carrier Type_3A. SELF PUBLIC              0
Carrier Type_4A. SELF PRIVATE             0
Carrier Type_5. SPECIAL FUND              0
County of Injury freq                     0
COVID-19 Indicator Enc                    0
District Name freq                        0
Gender Enc                      

**Accident Date**

Fill Year, Month and Day with median. Then recompute full date and from there fill missing values in Day of Week

In [122]:
# Define median values for Accident-related features from the training dataset
accident_med = {
    'Accident Date Year': round(df['Accident Date Year'].median()), 
    'Accident Date Month': round(df['Accident Date Month'].median()),
    'Accident Date Day': round(df['Accident Date Day'].median() ) 
}

Fill with median

In [123]:
df['Accident Date Year'].fillna(accident_med['Accident Date Year'], inplace=True)  
df['Accident Date Month'].fillna(accident_med['Accident Date Month'], inplace=True)
df['Accident Date Day'].fillna(accident_med['Accident Date Day'], inplace=True) 

test['Accident Date Year'].fillna(accident_med['Accident Date Year'], inplace=True)  
test['Accident Date Month'].fillna(accident_med['Accident Date Month'], inplace=True)
test['Accident Date Day'].fillna(accident_med['Accident Date Day'], inplace=True) 

Convert to Integers

In [124]:
df['Accident Date Year'] = df['Accident Date Year'].astype('Int64')
df['Accident Date Month'] = df['Accident Date Month'].astype('Int64')
df['Accident Date Day'] = df['Accident Date Day'].astype('Int64')

test['Accident Date Year'] = test['Accident Date Year'].astype('Int64')
test['Accident Date Month'] = test['Accident Date Month'].astype('Int64')
test['Accident Date Day'] = test['Accident Date Day'].astype('Int64')

Identify missing values and recompute Accident date to fill Day of the week

In [125]:
# select missing
missing_dayofweek_df = df['Accident Date Day of Week'].isnull()

missing_dayofweek_test = test['Accident Date Day of Week'].isnull()

In [126]:
# compute temp accident date for missing values in day of week
df.loc[missing_dayofweek_df, 'TEMP Accident Date'] = pd.to_datetime(
    df.loc[missing_dayofweek_df, ['Accident Date Year', 'Accident Date Month', 'Accident Date Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)


test.loc[missing_dayofweek_test, 'TEMP Accident Date'] = pd.to_datetime(
    test.loc[missing_dayofweek_test, ['Accident Date Year', 'Accident Date Month', 'Accident Date Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)

In [127]:
df.loc[missing_dayofweek_df, 'Accident Date Day of Week'] = df.loc[missing_dayofweek_df, 'TEMP Accident Date'].dt.dayofweek

test.loc[missing_dayofweek_test, 'Accident Date Day of Week'] = test.loc[missing_dayofweek_test, 'TEMP Accident Date'].dt.dayofweek

**C-2 Date**

Fill Year, Month and Day with median. Then recompute full date and from there fill missing values in Day of Week

In [128]:
# Create a dictionary with the median values of 'C-2 Year', 'C-2 Month', and 'C-2 Day' from the training set
c2_med = {
    'C-2 Date Year': round(df['C-2 Date Year'].median()),
    'C-2 Date Month': round(df['C-2 Date Month'].median()),
    'C-2 Date Day': round(df['C-2 Date Day'].median())
}

Fill with median

In [129]:
df['C-2 Date Year'].fillna(c2_med['C-2 Date Year'], inplace=True)  
df['C-2 Date Month'].fillna(c2_med['C-2 Date Month'], inplace=True)
df['C-2 Date Day'].fillna(c2_med['C-2 Date Day'], inplace=True) 

test['C-2 Date Year'].fillna(c2_med['C-2 Date Year'], inplace=True)  
test['C-2 Date Month'].fillna(c2_med['C-2 Date Month'], inplace=True)
test['C-2 Date Day'].fillna(c2_med['C-2 Date Day'], inplace=True) 

Convert to Integers

In [130]:
df['C-2 Date Year'] = df['C-2 Date Year'].astype('Int64')
df['C-2 Date Month'] = df['C-2 Date Month'].astype('Int64')
df['C-2 Date Day'] = df['C-2 Date Day'].astype('Int64')

test['C-2 Date Year'] = test['C-2 Date Year'].astype('Int64')
test['C-2 Date Month'] = test['C-2 Date Month'].astype('Int64')
test['C-2 Date Day'] = test['C-2 Date Day'].astype('Int64')

Identify missing values and recompute Accident date to fill Day of the week

In [131]:
# select missing
missing_dayofweek_df = df['C-2 Date Day of Week'].isnull()

missing_dayofweek_test = test['C-2 Date Day of Week'].isnull()

In [132]:
# compute temp accident date for missing values in day of week
df.loc[missing_dayofweek_df, 'TEMP C-2 Date'] = pd.to_datetime(
    df.loc[missing_dayofweek_df, ['C-2 Date Year', 'C-2 Date Month', 'C-2 Date Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)


test.loc[missing_dayofweek_test, 'TEMP C-2 Date'] = pd.to_datetime(
    test.loc[missing_dayofweek_test, ['C-2 Date Year', 'C-2 Date Month', 'C-2 Date Day']].astype(str).agg('-'.join, axis=1),
    errors='coerce'
)

In [133]:
df.loc[missing_dayofweek_df, 'C-2 Date Day of Week'] = df.loc[missing_dayofweek_df, 'TEMP C-2 Date'].dt.dayofweek

test.loc[missing_dayofweek_test, 'C-2 Date Day of Week'] = test.loc[missing_dayofweek_test, 'TEMP C-2 Date'].dt.dayofweek

Remove temporary variables

In [134]:
df.drop(['TEMP Accident Date', 'TEMP C-2 Date'], 
        axis = 1, inplace = True)

**C-3 Date**

Create a Binary variable:
- 0 if C-3 date is missing
- 1 if C-3 date exists

In [135]:
df['C-3 Date Binary'] = df['C-3 Date'].notna().astype(int)

test['C-3 Date Binary'] = test['C-3 Date'].notna().astype(int)

**First Hearing Date**

Create a Binary variable:
- 0 if First Hearing Date is missing
- 1 if First Hearing Date exists

In [136]:
df['First Hearing Date Binary'] = df['First Hearing Date'].notna().astype(int)

test['First Hearing Date Binary'] = test['First Hearing Date'].notna().astype(int)

**IME-4 Count**

Since IME-4 Count represents the number of IME-4 forms received per claim, we considered that a missing value represented 0 received forms, hence we will fill them with 0.

In [137]:
df['IME-4 Count'] = df['IME-4 Count'].fillna(0)

test['IME-4 Count'] = test['IME-4 Count'].fillna(0)

**Industry Code**

Assuming that a missing value in Industry Code represents an unknown code, it will be filled with 0.

In [138]:
df['Industry Code'] = df['Industry Code'].fillna(0)

test['Industry Code'] = test['Industry Code'].fillna(0)

**Drop Treated**

In [139]:
treated_drop = ['C-3 Date', 'First Hearing Date']

In [140]:
df.drop(treated_drop, axis = 1, inplace = True)

test.drop(treated_drop, axis = 1, inplace = True)

# 4. Export

<a href="#top">Top &#129033;</a>

In [147]:
df.to_csv('project_data/train_data_EDA.csv')
test.to_csv('./project_data/test_data_EDA.csv')